In [36]:
import yfinance as yf
import pandas as pd

def download_ticker_data(ticker, start_date, end_date):
    data = yf.download(ticker, start=start_date, end=end_date)

    # Flatten MultiIndex columns (if applicable)
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = [f'{col[0]}' for col in data.columns.values]

    # Reset the index to move 'Date' from index to a column
    data = data.reset_index()

    # Convert the 'Date' column to just the date part
    data['Date'] = data['Date'].dt.date

    return data


In [37]:
# Stock tickers and date range
ticker = 'SPY'

start_date = "2021-01-01"
end_date = "2024-01-01"

# Download stock data
ticker_data = download_ticker_data(ticker, start_date, end_date)

# Show the first few rows of the data
ticker_data.head()

[*********************100%***********************]  1 of 1 completed


,Date,Adj Close,Close,High,Low,Open,Volume
0,2021-01-04,349.471680,368.790009,375.450012,364.820007,375.309998,110210800
1,2021-01-05,351.878632,371.329987,372.500000,368.049988,368.100006,66426200
2,2021-01-06,353.982269,373.549988,376.980011,369.119995,369.709991,107997700
3,2021-01-07,359.241608,379.100006,379.899994,375.910004,376.100006,68766800
4,2021-01-08,361.288483,381.260010,381.489990,377.100006,380.589996,71677200


In [38]:
def get_interval_mapping(interval):

    interval_mapping = {
        "weekly": 5,
        "monthly": 20,
        "quarterly": 60,
        "biannually": 120
    }

    return interval_mapping.get(interval.lower(), "Invalid frequency")

In [39]:
def dollar_cost_average_strategy(data, interval, investment, comm_per_share, comm_min_per_order, comm_max_per_order, platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order):

    invested_amount = 0.0
    cash_balance = 0.0
    num_shares = 0.0
    original_cash = 0.0
    performance_data = []

    for index, row in data.iterrows():
        # Check that 'Close' is in the data columns
        # if 'Close' not in data.columns:
        #     raise KeyError("The 'Close' column is missing from the provided data. Available columns are: " + ", ".join(data.columns))
        # else:
        #     print("Available columns are: " + ", ".join(data.columns))
        date = row['Date']
        price = row['Close']
        price_bought_at = 0
        num_shares_bought = 0
        investing_amount = 0
        fees_incurred = 0

        # Increasing cash balance from pay check
        if index % 20 == 0 and index != 0:
            cash_balance += investment
            original_cash += investment

        # Decrease cash balance to invest
        if index % get_interval_mapping(interval) == 0 and cash_balance != 0:
            investing_amount = investment / \
                (20 / get_interval_mapping(interval))
            num_shares_rough = investing_amount / price
            fees_incurred = max(comm_min_per_order + platform_fee_min_per_order, min(max(num_shares_rough * (comm_per_share + platform_fee_per_share), comm_min_per_order +
                                                                                         platform_fee_min_per_order), (investing_amount) * (comm_max_per_order/100 + platform_fee_max_per_order/100)))
            cash_balance -= investing_amount
            invested_amount += investing_amount
            investing_amount -= fees_incurred
            num_shares += investing_amount / price
            price_bought_at = price
            num_shares_bought = investing_amount / price

        total = cash_balance + num_shares * price

        try:
            t_c = total / original_cash
        except ZeroDivisionError:
            t_c = 1

        performance_data.append((date, price_bought_at, num_shares_bought, fees_incurred, original_cash, cash_balance, invested_amount, num_shares * price, total,
                                t_c))

    performance_data = pd.DataFrame(performance_data, columns=['Date', 'Price Bought', 'Shares Bought', 'Fees Paid', 'Total Cash', 'Cash Balance', 'Total Invested',
                                                               'Investment Value', 'Portfolio Value', 'Portfolio/Cash'])

    performance_data['Performance'] = performance_data['Portfolio/Cash'] - 1

    return performance_data


In [40]:
def compare_interval(intervals, data, investment, comm_per_share, comm_min_per_order, comm_max_per_order, platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order):

    # Initialize an empty DataFrame to store the merged results
    result_df = pd.DataFrame()

    for interval in intervals:
        print(interval)
        output_data = dollar_cost_average_strategy(data, interval, investment, comm_per_share, comm_min_per_order, comm_max_per_order,
                                            platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order)
        filtered_data = output_data[["Date", "Total Invested", "Portfolio Value"]]

        # Rename columns to avoid overwriting during merges
        filtered_data = filtered_data.rename(columns={
            "Total Invested": f"Total Invested ({interval})",
            "Portfolio Value": f"Portfolio Value ({interval})"
        })

        # Merge `filtered_data` with `result_df` on "Date" column
        if result_df.empty:
            # First interval, initialize `result_df` with `filtered_data`
            result_df = filtered_data
        else:
            # Merge with existing data on "Date"
            result_df = pd.merge(result_df, filtered_data,
                                 on="Date", how="outer")

    # Sort the final result by Date, if necessary
    result_df = result_df.sort_values(by="Date").reset_index(drop=True)

    return result_df

ticker = "SPY"
start_date = "2021-01-01"
end_date = "2024-01-01"
ticker_data =  download_ticker_data(ticker, start_date, end_date)
print("Ticker data columns:", ticker_data.columns)

investment=100
comm_max_per_order = 0
comm_min_per_order = 0
comm_per_share = 0
platform_fee_max_per_order = 0
platform_fee_min_per_order = 0
platform_fee_per_share = 0
intervals = ["Weekly", "Monthly", "Quarterly", "Biannually"]
compare_data = compare_interval(intervals, ticker_data, investment, comm_per_share, comm_min_per_order, comm_max_per_order, platform_fee_per_share, platform_fee_min_per_order, platform_fee_max_per_order)

compare_data.head()


[*********************100%***********************]  1 of 1 completed

Ticker data columns: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
Weekly
Monthly
Quarterly
Biannually


,Date,Total Invested (Weekly),Portfolio Value (Weekly),Total Invested (Monthly),Portfolio Value (Monthly),Total Invested (Quarterly),Portfolio Value (Quarterly),Total Invested (Biannually),Portfolio Value (Biannually)
0,2021-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2021-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2021-01-06,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2021-01-07,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2021-01-08,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
